In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential, Model, models
from keras.initializers import Zeros
import tensorflow as tf
from tensorflow.keras.layers import Concatenate, Input, Bidirectional, LSTM, LSTMCell, Dense, Lambda, RepeatVector, Dot, Softmax, Reshape, Concatenate
from tensorflow.keras import activations

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
############# Hiperparâmetros #############
input_sequence_lenght = 30
input_vocab_size = 37
encoder_hidden_size = 32
decoder_hidden_size = 64
output_vocab_size = 11
output_sequence_lenght = 10
attention_size = 32

In [4]:
############# Declaração das redes #############
# Encoder:
encoder_inputs = Input(shape=(input_sequence_lenght, input_vocab_size), name="Input")
encoder_lstm = Bidirectional(LSTM(encoder_hidden_size, return_sequences=True, return_state=False), name="Encoder-LSTM")
encoder_outputs = encoder_lstm(encoder_inputs)

# Attention:
attention_dense = Sequential(
    [
        Dense(attention_size, activation=activations.tanh, use_bias=True),
        Dense(1, activation=None, use_bias=True)
    ], name="Attention-Dense"
)
# Decoder:
decoder_lstm = LSTMCell(encoder_hidden_size * 2, name="Decoder-LSTM-Cell")
decoder_dense = Dense(output_vocab_size, activation=activations.softmax, use_bias=False, name="Decoder-Dense")

concatenate = Concatenate(name="Concatenate")
repeat = RepeatVector(input_sequence_lenght, name="Repeat-Vector")
reshape = Reshape((input_sequence_lenght,), name="Reshape")
softmax = Softmax(name="Softmax")
dot = Dot(axes=1, name="Dot")
stack = Lambda(lambda x: tf.stack(x, axis=1), name="Stack")
############# Declaração das conexões entre as redes #############
encoder_outputs = encoder_lstm(encoder_inputs)

decoder_outputs = []
attention_wheights = []
zeros = tf.keras.layers.Lambda(lambda x: tf.zeros_like(x[:, 1, :]), name="Zeros")(encoder_outputs)
decoder_states = [zeros, zeros]
for i in range(output_sequence_lenght):
    # Mecanismo de Atenção
    attention_input = concatenate([repeat(decoder_states[0], ), encoder_outputs])
    attention_wheight = reshape(attention_dense(attention_input))
    attention_wheight = softmax(attention_wheight)
    attention_wheights.append(attention_wheight)
    context_vector = dot([attention_wheight, encoder_outputs])
    lstm_output, decoder_states = decoder_lstm(context_vector, states=decoder_states) # Célula LSTM
    decoder_outputs.append(decoder_dense(lstm_output)) # Dense com softmax

model_output = stack(decoder_outputs)
attention_output = stack(attention_wheights)

############# Declaração do modelo #############
model = Model(encoder_inputs, model_output)
attention_model = Model(encoder_inputs, attention_output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
############# Import de auxiliares #############
# São funções para formatação em vetores one-hot e para download do dataset

import os
if not os.path.exists("auxfunctions.py"):
  !wget https://raw.githubusercontent.com/LipeAma/Phrase2Date/refs/heads/main/auxfunctions.py
from auxfunctions import phrase2hotvec, date2hotvec, hotvec2phrase, hotvec2date, dataset

dataset[:10] # Exemplo do dataset

--2024-11-28 01:56:24--  https://raw.githubusercontent.com/LipeAma/Phrase2Date/refs/heads/main/auxfunctions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1792 (1.8K) [text/plain]
Saving to: ‘auxfunctions.py’

auxfunctions.py     100%[===================>]   1.75K  --.-KB/s    in 0s      

2024-11-28 01:56:24 (31.3 MB/s) - ‘auxfunctions.py’ saved [1792/1792]



[['9 may 1998', '1998-05-09'],
 ['10.11.19', '2019-11-10'],
 ['9/10/70', '1970-09-10'],
 ['saturday april 28 1990', '1990-04-28'],
 ['thursday january 26 1995', '1995-01-26'],
 ['monday march 7 1983', '1983-03-07'],
 ['sunday may 22 1988', '1988-05-22'],
 ['08 jul 2008', '2008-07-08'],
 ['8 sep 1999', '1999-09-08'],
 ['thursday january 1 1981', '1981-01-01']]

In [6]:
def predict(src):
  src = phrase2hotvec(src, input_sequence_lenght)
  pred = model.predict(np.array([src]))[0]
  pred = hotvec2date(pred)
  return ''.join(pred)

In [7]:
def attention_map(src):
    f = plt.figure(figsize=(25, 8))
    ax = f.add_subplot(1, 1, 1)
    ax.set_xlabel('Input')
    ax.set_ylabel('Output')

    src = phrase2hotvec(src, input_sequence_lenght)

    ax.set_xticks(range(input_sequence_lenght))
    ax.set_xticklabels(hotvec2phrase(src))

    src = np.array([src])
    attention_map = attention_model.predict(src)[0]
    pred = model.predict(src)[0]
    pred = hotvec2date(pred)

    i = ax.imshow(attention_map, interpolation='nearest', cmap='Blues')

    cbaxes = f.add_axes([0.2, 0, 0.6, 0.05])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Pesos calculados pelo mecanismo de atenção', labelpad=2)

    ax.set_yticks(range(output_sequence_lenght))
    ax.set_yticklabels(pred)
    return f

In [8]:
############# Organização do dataset #############
source, target = list(zip(*[(phrase2hotvec(phrase, input_sequence_lenght), date2hotvec(date))  for (phrase, date) in dataset]))
source = np.array(source)
target = np.array(target)
print("Sources shape:", source.shape)
print("Source example:\n", source[0])
print("\nTargets shape:", target.shape)
print("Target example:\n", target[0])

Sources shape: (10000, 30, 37)
Source example:
 [[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]

Targets shape: (10000, 10, 11)
Target example:
 [[0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]]


In [9]:
from math import ceil
dataset_size = len(dataset)
training_ratio = 0.8
training_size = ceil(dataset_size * training_ratio)
X_train = source[:training_size]
y_train = target[:training_size]
X_test = source[training_size:]
y_test = target[training_size:]
batch_size = 80
resultados = []

In [10]:
############# Verificando Acurácia #############
def acuracia():
  total_correct_chars = 0
  total_correct_dates = 0
  results = model.predict(source[:training_size])
  for i in range(training_size, dataset_size):
    src = source[i]
    trg = dataset[i-training_size][1]
    pred = results[i-training_size]
    pred = hotvec2date(pred)

    correct_chars = sum([c1 == c2 for (c1,c2) in zip (pred, trg)])
    total_correct_chars += correct_chars
    if correct_chars == 10: total_correct_dates += 1

  dates_percentage = total_correct_dates / (dataset_size - training_size)
  chars_percentage = total_correct_chars / ((dataset_size - training_size)*10)
  return dates_percentage, chars_percentage


In [11]:
import csv
def export_list_matrix_to_csv(matrix, filename):
  with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(matrix)

export_list_matrix_to_csv(resultados, f'{encoder_hidden_size}-{decoder_hidden_size}-{attention_size}.csv')

In [12]:
############# Treinamento: total = 1000 épocas #############
testes_para_attention_map = [8750, 9000, 9500, 8145, 9421] # qualquer um vale
testes_para_attention_map = [dataset[i][0] for i in testes_para_attention_map]
print(testes_para_attention_map)
for n in range(100):
  print("epoch:", n)
  if n in [0, 10, 50, 100, 500, 1000]:
    for i in range(len(testes_para_attention_map)):
      fig = attention_map(testes_para_attention_map[i])
      fig.savefig(f"({encoder_hidden_size}-{decoder_hidden_size}-{attention_size})-epoch{n}-sentence{i}.png", bbox_inches="tight")
      plt.close(fig)
  history = model.fit(X_train, y_train, epochs=1, batch_size=batch_size)
  resultados.append((history.history["accuracy"][0], *acuracia()))
  if n%20 == 0: export_list_matrix_to_csv(resultados, f'({encoder_hidden_size}-{decoder_hidden_size}-{attention_size})-{n}.csv')

['february 26 1981', 'thursday january 18 2018', '16 dec 1982', '6/22/89', 'monday january 23 2017']
epoch: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.2211 - loss: 2.2078
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
epoch: 1
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4215 - loss: 1.6855
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
epoch: 2
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6009 - loss: 1.0690
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
epoch: 3
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6573 - loss: 0.9235
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
epoch: 4
100/100 ━

In [14]:
# prompt: make a line graph from the columns of a csv file

import pandas as pd
import matplotlib.pyplot as plt

# Assuming your CSV file is named 'results.csv' and is in the current directory.
# Replace 'results.csv' with the actual filename if it's different.
try:
  df = pd.read_csv('(64-32-10)-100.csv', header=None, names=['accuracy', 'dates_percentage', 'chars_percentage'])

  # Create the line graph
  plt.figure(figsize=(10, 6))
  plt.plot(df['accuracy'], label='Precisão no dataset de treino')
  plt.plot(df['dates_percentage'], label='Precisão no dataset de teste')
  plt.plot(df['chars_percentage'], label='Precisão no dataset de teste (caractere-por-caractere)')

  # Customize the plot
  plt.xlabel('Época')
  plt.ylabel('Precisão')
  plt.title('Performance do modelo conforme as épocas')
  plt.legend()
  plt.grid(True)

  # Show the plot
  plt.show()

except FileNotFoundError:
  print("Error: 'results.csv' not found in the current directory.")
except pd.errors.ParserError:
    print("Error: Could not parse the CSV file. Please check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error: 'results.csv' not found in the current directory.
